Testbed for bots, by Arseny, based on Henry's testing_grounds.

In [1]:
import numpy as np
import pandas as pd
from operator import itemgetter
from copy import deepcopy
import json

import draftsimtools as ds

In [2]:
# Load M19 drafts
raw_drafts = ds.load_drafts("../../data/m19_2.csv")

# m19_set = ds.create_set("data/m19_rating.tsv", "data/m19_land_rating.tsv")

In [15]:
with open('../../data/Allsets.json', 'r',encoding='utf-8') as json_data:
    mtgJSON = json.load(json_data)
    
jsonSubset = mtgJSON['M19']['cards']

thisSet = {card['name'] : card for card in jsonSubset}
nameList = pd.DataFrame(thisSet.keys(),columns = ['Name']) # Instead of relying on cvs, get names from json

In [23]:
nameList['Name'] = nameList.Name.str.replace(' ','_')
nameList.Name[nameList.Name.str.find(',')!=-1] # List of all names with a comma

12       Ajani,_Adversary_of_Tyrants
13             Ajani,_Wise_Counselor
19           Arcades,_the_Strategist
40             Chromium,_the_Mutable
106    Goreclaw,_Terror_of_Qal_Sisma
136           Lathliss,_Dragon_Queen
138          Lena,_Selfless_Champion
146      Liliana,_Untouched_by_Death
147         Liliana,_the_Necromancer
172          Nicol_Bolas,_the_Arisen
173         Nicol_Bolas,_the_Ravager
183        Palladia-Mors,_the_Ruiner
215           Sai,_Master_Thopterist
220              Sarkhan,_Dragonsoul
221               Sarkhan,_Fireblood
264        Tezzeret,_Artifice_Master
265        Tezzeret,_Cruel_Machinist
281       Vaevictis_Asmadi,_the_Dire
Name: Name, dtype: object

In [24]:
# This utility method searches for "Name" column in nameList that have commas
nameList, raw_drafts = ds.fix_commas(nameList, raw_drafts) # Returns a tuple, as it updates both
nameList.Name[nameList.Name.str.find(',')!=-1] # There should be no longer any cards with commas

Series([], Name: Name, dtype: object)

In [25]:
# Process the drafts, deconstructing packs (hands) at every turn of every draft
drafts = ds.process_drafts(raw_drafts)

Processing draft: 0.
Processing draft: 10000.
Processing draft: 20000.
Processing draft: 30000.
Processing draft: 40000.
Processing draft: 50000.
Processing draft: 60000.
Processing draft: 70000.
Processing draft: 80000.
Processing draft: 90000.
Processing draft: 100000.


In [26]:
# Splits into (toy) training and test sets. NOTE: For real training, use all drafts.
subset_drafts = drafts[5000:5500]
#train, test = train_test_split(subset_drafts, test_size = 0.4)

In [27]:
raw_drafts[1:100]

'200002,M19,Sai_Master_Thopterist,Gearsmith_Guardian,Frilled_Sea_Serpent,Gearsmith_Guardian,Field_Cr'

### Now this part below will just create weird bots and test them

But for now I can as well sketch a random bot here.

In [ ]:
# Creates a bot that implements current Draftsim logic
# draftsim_bot = ds.DraftsimBot(rating_dict)

In [29]:
# nameList.Name = nameList.Name.str.lower()
nCardsInSet = len(nameList)
nCardsInSet

300

In [31]:
nameList.Name[nameList.Name.str[0]=='T']

258              Take_Vengeance
259          Talons_of_Wildwood
260              Tattered_Mummy
261               Tectonic_Rift
262      Tezzeret's_Gatebreaker
263          Tezzeret's_Strider
264    Tezzeret_Artifice_Master
265    Tezzeret_Cruel_Machinist
266            Thorn_Lieutenant
267            Thornhide_Wolves
268                        Thud
269                Timber_Gorge
270              Titanic_Growth
271            Tolarian_Scholar
272            Tormenting_Voice
273                Totally_Lost
274            Tranquil_Expanse
275        Transmogrifying_Wand
276               Trumpet_Blast
277            Trusty_Packbeast
278           Two-Headed_Zombie
Name: Name, dtype: object

In [41]:
# Count-hot encoding
for iDraft in range(11,12): #range(len(subset_drafts)):
    draft = subset_drafts[iDraft]
    counter = 0
    for pack in draft:
        counter += 1        
        print(counter)
        hand = np.zeros(nCardsInSet+1)
        for card in pack:
            cardName = card
            try:
                pos = nameList[nameList.Name==cardName].index[0]
            except:
                pos = nCardsInSet # Use this very large number (300 for M19) if not found; for now
                # print("---Unrecognized card:") # All unrecognized cards here seem to be foil lands, so ignore 'em
            hand[pos] += 1
            # print(cardName,pos)
        # print(pack)
        for i in hand:
            print("%d" % i, end="")
        print()

1
0000000000000000000000000000100010000000000010000000000000000000000000000010000000000000000010000000000000000000000000000000000000000100100000000000000000010000000000000000000000010000000000010000000000000000000000100000000000000000000000000000010000000000000001000000000000001000000000000000000000001
2
0000000000000000000000000000000000001000000000000000000000000000000000000000000100000000000000000010000010000000000001000000000000000000000000000000000000000000000000000000000000010000000000000000001000000000000000000000000000000000000000000101001000000000000000000000000000010000000000000000000110001
3
0000000000000000000000000010000000000000000000000000000000000110000000000000000000000000000100000000000000000001000000100100000000000100000000000000000100000000000000000000000010000000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000001000000000000000001000000
4
00000000000000000000000000000000001000000000001000000000000000000000000000000000000000

0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001
30
0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
31
0000000000000010100000000000000001000000000000000000000000000000100000000000001000000000000010000000000000000000000000000101001000000000000000000000000000000000000000000000000000000000010000000000100000000000000000000000000000000000000000000000001000000100000000000001000000000000000000000000000000001
32
0000000000000000000000000000000000001000000000000000000000000000100000000000010000000

In [ ]:
# Gets bot accuracy for some packs
tester = ds.BotTester(test)
tester.evaluate_bots([draftsim_bot, sgd_bot], ["Draftsim2018", "Draftsim2018SGD"])
tester.write_evaluations()